In [1]:
import os
import torch
from torch import nn
import torchvision as tv
import torchvision.transforms.v2 as v2
from torch.utils.data import Dataset, DataLoader, random_split
from icecream import ic
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
from tqdm import tqdm # Progress bar for training

from tensorboardX import SummaryWriter

from dataLoading import CIFAR10Dataset, displayImageGrid

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"


def validateModelIO(model:nn.Module):
    
    print(f"Using device: {device}")

    model = model.to(device)

    print(model)
    print(f"Model has {sum(p.numel() for p in model.parameters())} parameters.")
    print(summary(model, input_size=(3, 32, 32)))

    dummy_input = torch.randn(1, 3, 32, 32, device=device, dtype=torch.float)
    output = model(dummy_input)
    assert output.size() == (1, 10), f"Expected output size (1, 10), got {output.size()}!"
    print("Test passed!")



In [50]:
class CNN(nn.Module):
    """
    A simple CNN for classifying images in the AnimalDataset.
    """

    def __init__(self):

        super().__init__()
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=32, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),

            nn.Flatten(),
            
            nn.Linear(32 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.Softmax(dim=0)
        )
        

    def forward(self, x):
        
        # for layer in self.fc:
        #     x = layer(x)
        #     print(type(x).__class__.__name__)
        #     print(x.size())
        
        x = self.fc(x)
        
        return x




class ResidualBlock(nn.Module):
    
    printOutsize = False
    
    def __init__(self, channelCount, activation:nn.Module=nn.ReLU(), kernel_size=3, stride=1, padding=1):
        super().__init__()
        
        self.c1 = nn.Sequential(
            nn.Conv2d(in_channels=channelCount, out_channels=channelCount, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.ReLU()
        )
        
        self.c2 = nn.Sequential(
            nn.Conv2d(in_channels=channelCount, out_channels=channelCount, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.ReLU()
        )
        
        self.activation = activation
        

    def forward(self, x):
        
        if self.printOutsize:
            print(x.size())
            
        y1 = self.c1(x)
        
        if self.printOutsize:
            print(y1.size())
            
        y = self.c2(y1)
        
        if self.printOutsize:
            print(str(y.size()) + '\n')
            
        y = y + x
        
        self.outsize = y.size()
        
        return self.activation(y)



class ResidualCNN(nn.Module):
    def __init__(self, fc:nn.Sequential, printOutsize=False):

        super().__init__()
        
        ResidualBlock.printOutsize = printOutsize
        
        self.fc = fc
        

    def forward(self, x):
        """
        Forward pass of the model.
        Apply the layers defined in `__init__` in order.
        Args:
            x (Tensor): The input tensor of shape (N, 3, 256, 256).
        Returns:
            output (Tensor): The output tensor of shape (N, 10).
        """
        
        x = self.fc(x)
        
        return x
    pass



In [51]:

fc1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),

            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Flatten(),
            nn.Linear(in_features=144, out_features=10),
            nn.Softmax(dim=0)
        )


fc1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),

            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Flatten(),
            nn.Linear(in_features=144, out_features=10)
        )

# TODO: Change this
fc2 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),

            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            ResidualBlock(channelCount=16),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            
            nn.Flatten(),
            nn.Linear(in_features=144, out_features=10)
        )

In [52]:

validateModelIO(ResidualCNN(fc=fc1, printOutsize=True))


Using device: cuda
ResidualCNN(
  (fc): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ResidualBlock(
      (c1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
      (c2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
      (activation): ReLU()
    )
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): ResidualBlock(
      (c1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
      (c2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=

In [40]:


# Create dataset instances
fullDataset = CIFAR10Dataset(rootDirectory='cifar-10', csvFilename='trainLabels.csv', dataFolder='train', transform=None)
TRAIN_DATASET, VALIDATION_DATSET, TEST_DATSAET = random_split(fullDataset, [0.8, 0.1, 0.1])

BATCH_SIZE = 256
trainLoader = DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE, shuffle=False)

mean = 0.
std = 0.
for images, _ in trainLoader:
    
    batchSamples = images.size(0)
    # Get an image view of shape (batchSamples, C, H*W) which is faster than a transpose as we don't shift any data
    images = images.view(batchSamples, images.size(1), -1)
    # Calculate total mean and total std over dim=2
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)

# Divide means and stdevs by number of samples
mean /= len(trainLoader.dataset)
std /= len(trainLoader.dataset)
print(mean)
print(std)


tensor([0.4919, 0.4827, 0.4468])
tensor([0.2024, 0.1995, 0.2011])


In [53]:
autoAugmentCIFAR = tv.transforms.Compose([
    v2.AutoAugment(v2.AutoAugmentPolicy.CIFAR10),
])

# Update transform with normalized values
transform = tv.transforms.Compose([
    CIFAR10Dataset.defaultTransform,
    # Add other transforms here
    v2.Normalize(mean=mean, std=std)
])

# Create dataset instances
fullDataset = CIFAR10Dataset(rootDirectory='cifar-10', csvFilename='trainLabels.csv', dataFolder='train', transform=transform)
TRAIN_DATASET, VALIDATION_DATSET, TEST_DATSAET = random_split(fullDataset, [0.8, 0.1, 0.1])

BATCH_SIZE = 512
trainLoader = DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE, shuffle=True)

In [57]:
def trainEpoch(model:nn.Module, dataloader:DataLoader, optimizer:torch.optim):
    
    lossFunction = nn.CrossEntropyLoss()
    
    totalLoss = 0
    
    N = 0
    correct = 0
    
    for features, labels in dataloader:
        x, y = features.to(device), labels.to(device)

        optimizer.zero_grad()
        
        forwardPass = model.forward(x)
        
        # This adds the current accuracy to correct which is averaged over all iterations of the epoch.
        correct += (forwardPass.argmax(dim=1) == y).float().mean().item()

        loss = lossFunction(forwardPass, y)
        totalLoss += loss.item()
        loss.backward()
        
        optimizer.step()
        
        N += 1
    
    return totalLoss / N, correct / N
        





EPOCHS = 60
RUN = '1-ResNetfc1'
RUNS_DIR = 'runs'

lr = 1e-3
gamma = 0.9
model = ResidualCNN(fc=fc1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)


writer = SummaryWriter(f'{RUNS_DIR}/run{RUN}_batch{BATCH_SIZE}_lr{lr:0.6f}', flush_secs=10)

pbar = tqdm(range(EPOCHS))
for epoch in pbar:
    trainLoss, trainAccuracy = trainEpoch(model=model, dataloader=trainLoader, optimizer=optimizer)

    pbar.set_description("trainLoss: {:.4f}, trainAccuracy: {:.4f}".format(trainLoss, trainAccuracy), refresh=True)
    writer.add_scalar('trainLoss', trainLoss, epoch)
    writer.add_scalar('trainAccuracy', trainAccuracy, epoch)
    writer.add_scalar('lr', scheduler.get_last_lr(), epoch)

    scheduler.step()



# pbar = tqdm(range(EPOCHS))
# for epoch in pbar:
#     train_loss, train_acc = trainEpoch(model=model, dataloader=trainLoader, optimizer=optimizer)
#     # val_loss, val_acc = self.val_epoch()
#     # self.writer.add_scalar('lr', self.lr_schedule.get_last_lr(), epoch)
#     # self.writer.add_scalar('val_acc', val_acc, epoch)
#     # self.writer.add_scalar('val_loss', val_loss, epoch)
#     # self.writer.add_scalar('train_acc', train_acc, epoch)
#     writer.add_scalar('train_loss', train_loss, epoch)
#     # pbar.set_description("val acc: {:.4f}, train acc: {:.4f}".format(val_acc, train_acc), refresh=True)
#     # if val_acc > best_val_acc:
#     #     best_val_acc = val_acc
#     #     best_epoch = epoch
#     #     torch.save(self.model.state_dict(), model_file_name)
#     # self.lr_schedule.step()
#     scheduler.step()




trainLoss: 0.5924, trainAccuracy: 0.7930: 100%|██████████| 60/60 [25:38<00:00, 25.65s/it]
